## Create a vector database from embeddings

Once the algorithm found the optimized embeddings, let's put those into a vector database for easier management and similarity search.

The vector database creation or updating is added to `scripts/train.py`.

## Imports

In [1]:
import faiss
import numpy as np
from fastai.collab import load_learner

## Create vector db

In [2]:
learn = load_learner("../api/learner.pkl")

In [ ]:
vector_dimension = vectors.shape[1]
index = faiss.IndexFlatL2(vector_dimension)
faiss.normalize_L2(vectors)
index.add(vectors)

_vector = np.array([search_vector])
faiss.normalize_L2(_vector)

k = index.ntotal
distances, ann = index.search(_vector, k=k)

results = pd.DataFrame({'distances': distances[0], 'ann': ann[0]})